# Setup

In [3]:
import wbgapi as wb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import random
import textwrap as twp
from matplotlib import cm
import matplotlib.colors
import translators as ts
import matplotlib.colors as colors


from turtle import color
plt.rcParams['font.family'] = 'Manjari'
# plt.style.use("Solarize_Light2")


Using state  server backend.


# WB Indicator Generator Random

In [4]:
def indicator_generator(database_id):

    # select database
    print("db_id: ", database_id)
    wb.db = database_id

    indicator_list = [i[0] for i in wb.series.info().table()]
    indicator = random.choice(indicator_list)
    print("indicator: ", indicator)


    #defualt back to WDI database
    wb.db = 2

    return indicator

###
indicator_generator(random.choice([1,2,3,5,6]))

db_id:  3
indicator:  RL.PER.RNK.LOWER


'RL.PER.RNK.LOWER'

# Plot

In [5]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [2]:
def plot_indicator(indicator, economy):

    try:
        df = wb.data.DataFrame(indicator, economy, skipBlanks=True, numericTimeKeys=True)
        df = df.T[economy]
        df.index = pd.to_datetime(df.index, format = "%Y").year
        df = df.dropna().round(1)
    except:
        print("ERROR! Initial DF did not load. Starting over...\n\n")
        plot_indicator(indicator_generator(2), economy)
        raise ValueError('Initial DF did not load')
    # entries
    total_years = len(df)
    print("total years", total_years)

    if total_years <=5:
        print("ERROR! Too few years. Starting over...\n\n")
        plot_indicator(indicator_generator(2), economy)
        return ValueError('Too few years.')


    # FIG
    fig, axs = plt.subplots(1,1, figsize=(14,7), dpi=200)
    title = wb.series.info(id=indicator).table()[0][1]
    title_plus = f"\n{ts.google(title, from_language='en', to_language='es')}"
    title_plus = "Bolivia: " + twp.fill(title_plus.replace('\$\$','\\$\\$'), 42)
    fig.suptitle(title_plus, fontsize= 33, y=.963, va="bottom", color="#2e2e2e")
    
    #bar colors
    CM = random.choice([cm.Greens, 
                        cm.Reds, 
                        cm.viridis_r, 
                        cm.magma_r,
                        cm.inferno_r,
                        cm.cool,
                        cm.winter,
                        cm.spring_r,
                        ])
    # display(CM)

    CM = truncate_colormap(CM, 0.21, .9)
    # display(CM)
    colors = CM(df.values / float(max(df.values)))# + df.values / float(max(df.values))/2)

    # plot bar
    bar = axs.bar(df.index, df, color=colors, zorder=6)
    if total_years <= 33:
        axs.bar_label(bar)

    # fig params
    fig.set_facecolor("#f4efe7")#f4efe7
    axs.set_facecolor("#f5f5f5")
    axs.grid(lw=.5, ls=":", zorder=1)
    axs.ticklabel_format(style='plain', useOffset=False)
    
    #series info
    indicator_info = wb.series.metadata.get(indicator)
    source = indicator_info.metadata["Source"]
    source = source.replace('\$\$','\\$\\$') + f" Traducciones vía Google Translate API // wbgapi indicator: {indicator})"
    source = "Fuente: " + f"{twp.fill(ts.google(source, from_language='en', to_language='es'),60)}"
    fig.text(.288, 0.06, source + "\n\nVisualización de BoliGráfica", ha="center", va="top", bbox={"alpha":.2, "color": colors[int(total_years/5)]}, fontsize=11)
    try:
        Longdefinition = indicator_info.metadata["Longdefinition"]
        Longdefinition = ts.google(Longdefinition, from_language='en', to_language='es')
        Longdefinition = "Descripción: " + twp.fill(Longdefinition, 75)
        Longdefinition = Longdefinition.replace('\$\$','\\$\\$')
        fig.text(.683, 0.06, Longdefinition, ha="center", va="top", bbox={"alpha":.2, "color": colors[int(total_years/4)]})
    except:
        ValueError("Longdefinition was not found or smth else")

    # savefig
    filename = indicator.replace(".", "-")
    display(filename)
    fig.savefig(f"figures/bo_{filename}", bbox_inches = "tight")
    return fig, title_plus


#### TEST

# plot_indicator(indicator_generator(database_id = 2), economy="BOL")
# pass

In [1]:
# for i in range(300):
#     try:
#         plot_indicator(indicator_generator(2), economy="BOL")
#     except:
#         pass